Assume the following
1. Since this is for Spatial Data - typical importing text or whatever is unnecessary. The data used in a collection of tweets which have x, y and t values
    {Twts} = {t1_{x1,y1,t1}, t2_{x2,y2,t2}.... tn_{xn,yn,tn}}
    where ti_{xi,yi,ti} is tweet i with corresponding longitude, latitude and timestamp it was posted.
2. Tweets cover only the small region on Phoenix AZ USA as the vocab creation would be too large for anything larger.
3. There are video, audio, text embedding vector models. Spatial data is different from text data which is one dimensional.
4. We therefore do the following -
    use s2sphere and convert lat/lng to S2 Cell ID at Level 24 (~1m resolution), Round latitude, longitude to 5 decimal places, and Generate S2 Cell IDs. These will act as base tokens for our vocabulary.
5. Twitter data downloaded from UCR-star


In [43]:
#simple cuda check

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Yes! CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("❌ CUDA not available. Using CPU.")

# Create two tensors and move them to the selected device
a = torch.rand(3, 3).to(device)
b = torch.rand(3, 3).to(device)

# Do a simple GPU operation
c = a + b

# Print result and device info
print("Tensor a:", a)
print("Tensor b:", b)
print("Sum c:", c)
print("Result stored on:", c.device)


✅ Yes! CUDA is available. Using GPU: NVIDIA GeForce RTX 2070
Tensor a: tensor([[0.4573, 0.3813, 0.5573],
        [0.9191, 0.2753, 0.4590],
        [0.8323, 0.1355, 0.5499]], device='cuda:0')
Tensor b: tensor([[0.4963, 0.6937, 0.7964],
        [0.5391, 0.0639, 0.3868],
        [0.2575, 0.3230, 0.5140]], device='cuda:0')
Sum c: tensor([[0.9536, 1.0750, 1.3538],
        [1.4582, 0.3393, 0.8458],
        [1.0899, 0.4584, 1.0639]], device='cuda:0')
Result stored on: cuda:0


In [44]:
import pandas as pd
df = pd.read_csv('data/output.csv', usecols=['S2_Cell_ID'])
print(df)
df.to_csv('data/S2_Cell_ID.csv', index=False)
column_data = pd.read_csv('data/S2_Cell_ID.csv')
unique_values = column_data.drop_duplicates()
unique_values.to_csv('data/uniqueS2s.csv', index=False)

                  S2_Cell_ID
0       12686489114809790464
1       11913439229996199936
2       12687520983359254528
3       12686489262385393664
4       12687525980826963968
...                      ...
967084  11831257354543017984
967085  11831260383644659712
967086  12711443012489760768
967087  11831357078610014208
967088  12006089755876593664

[967089 rows x 1 columns]


In [18]:
vocab_tokens = list(df['S2_Cell_ID'].astype(str).str.strip())
print(vocab_tokens[:20])
print("Total length: ", len(vocab_tokens))
vocab_tokens = list(set(vocab_tokens))
print(vocab_tokens[:20])
print("Total length:", len(vocab_tokens))

[12686489114809790464, 11913439229996199936, 12687520983359254528, 12686489262385393664, 12687525980826963968, 11831258441354145792, 11831048230084284416, 12687451638397939712, 12006417556676259840, 12687475888992038912, 12687517951542562816, 12006441324348600320, 12687517932484734976, 12687525980826963968, 12687480861127626752, 11831256175924252672, 12687522130192519168, 12687524198988075008, 11831253183804510208, 11831258441354153984]
Total length:  967089
[11831247360713830400, 11892204956713504768, 12687527026841374720, 11831244104856276992, 12006515941861806080, 12687479919901995008, 12687477355806519296, 11831242287006158848, 12687542303495360512, 12686534844288421888, 12687480786406436864, 12687305072045297664, 11831255869295546368, 12006538499019624448, 12687480108870135808, 12708071581439889408, 12006435355280936960, 12683695488549392384, 12686493218434846720, 11892209674620375040]
Total length: 672888


In [24]:
import s2sphere
import pandas as pd

# Function to convert lat/lng to S2 Cell ID at Level 24 (~1m resolution)
def latlng_to_cellid(lat, lng, level=16):
    """Convert latitude and longitude to an S2 cell ID at the given level."""
    latlng = s2sphere.LatLng.from_degrees(lat, lng)
    cell = s2sphere.CellId.from_lat_lng(latlng).parent(level)
    return cell.id()


# File paths (Modify as needed)
input_file = "data/output.csv"  # Path to input file

# Read only the first two columns (Latitude, Longitude) and force float conversion
df = pd.read_csv(input_file, usecols=[0, 1], names=["Latitude", "Longitude"], header=0)

# Round to 5 decimal places
df["Latitude"] = df["Latitude"].astype(float).round(5)
df["Longitude"] = df["Longitude"].astype(float).round(5)

# Generate S2 Cell ID
df["S2_Cell_ID"] = df.apply(lambda row: latlng_to_cellid(row["Latitude"], row["Longitude"]), axis=1)

print(len(df["S2_Cell_ID"]))

967089


In [25]:
vocab_tokens = list(df['S2_Cell_ID'].astype(str).str.strip())
print(vocab_tokens[:20])
print("Total length: ", len(vocab_tokens))
vocab_tokens = list(set(vocab_tokens))
print(vocab_tokens[:20])
print("Total length:", len(vocab_tokens))

['12686489114824933376', '11913439230044405760', '12687520983402151936', '12686489262464434176', '12687525980596600832', '11831258441576349696', '11831048230307627008', '12687451638470803456', '12006417556630929408', '12687475888929898496', '12687517951692111872', '12006441324443074560', '12687517932364759040', '12687525980596600832', '12687480860891414528', '11831256175981101056', '12687522130158419968', '12687524198722043904', '11831253183999508480', '11831258441576349696']
Total length:  967089
['12686487810765488128', '12687453004270403584', '11831244530714148864', '11913548339930464256', '12711823569441521664', '12686080581830705152', '12708404214980673536', '11831254166473277440', '12687481052554330112', '11919199236454875136', '12687537832021983232', '12687479480596299776', '11912188011581800448', '12687537292466716672', '12687542560244105216', '11831355879351910400', '11831247849650126848', '12686106769320050688', '11831244142019608576', '12687452989774888960']
Total length: 10

In [26]:
# Save only required columns
df[["Latitude", "Longitude", "S2_Cell_ID"]].to_csv('data/S2Cell_ID_level16.csv', index=False)

In [27]:
with open('data/uniqueS2s_level16.csv', 'w') as file:
    # Write each item in the list on a new line
    for item in vocab_tokens:
        file.write(f"{item}\n")

In [28]:
# Clearing all variables
%reset -f

In [45]:
import pandas as pd
df = pd.read_csv('data/S2Cell_ID_level16.csv', usecols=[2],names=["S2_Cell_ID"], header=0)
data = list(map(int, df['S2_Cell_ID'].astype(str).str.strip()))
df = pd.read_csv('data/uniqueS2s_level16.csv', names=["Token"])
vocab_tokens = list(map(int, df['Token'].astype(str).str.strip()))
del df
print(len(data), len(vocab_tokens))

# Build lookup dict
id_to_idx = {token: idx for idx, token in enumerate(vocab_tokens)}

# Map data to indices
indexed_data = [id_to_idx[cell_id] for cell_id in data if cell_id in id_to_idx]

print("Sample data:", data[:5])
print("Sample vocab_tokens:", vocab_tokens[:5])

print("Example data[0] in vocab?", data[0] in vocab_tokens)

print("Number of matched tokens:", len(indexed_data))

967089 103244
Sample data: [12686489114824933376, 11913439230044405760, 12687520983402151936, 12686489262464434176, 12687525980596600832]
Sample vocab_tokens: [12686487810765488128, 12687453004270403584, 11831244530714148864, 11913548339930464256, 12711823569441521664]
Example data[0] in vocab? True
Number of matched tokens: 967089


In [39]:
import torch
# safe to convert to tensor
data_tensor = torch.tensor(indexed_data, dtype=torch.long)
print(data_tensor[:100])

tensor([ 56442, 101734, 102200,  37046,  81968,  81295,  30439,  55325,  38104,
         27681,  23271,  84760,  94323,  81968,  21692,  45532,  98796,  42377,
         31790,  81295,  87385,  90341,  61198,  32257,  65059,  82328,  90244,
         44086,  23396,  11446,  63504,  28649,   9696,  64578,  28878,  59931,
         11034,  82280,  20143,  68886,   9007,  94323,  71874,  69583,  10508,
         39779,  68886,  85930,  77214,  89054,   1326,  24350,  79486,  14454,
          2697,  62931,  77214,  66360,  71851,   4057,  74818,  73539,  80809,
         92655,  84091,  59471, 102126,  75528, 100902,  75528,   4057,  16055,
         16446,  15631,  19860,  52816,  10508,  11370,  21657,  79697, 100883,
        103172,  40130,  99893,  90612,  47054,  22951,  26619,  87251,  73312,
         82005,  36274,  35970,  11552,  16950,  80005,  96868,  78875,  90934,
         49316])


In [40]:
n = int(0.8 * len(data_tensor))
train_data = data_tensor[:n]
val_data = data[n:]

In [41]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("input", context, "target", target)

input tensor([56442]) target tensor(101734)
input tensor([ 56442, 101734]) target tensor(102200)
input tensor([ 56442, 101734, 102200]) target tensor(37046)
input tensor([ 56442, 101734, 102200,  37046]) target tensor(81968)
input tensor([ 56442, 101734, 102200,  37046,  81968]) target tensor(81295)
input tensor([ 56442, 101734, 102200,  37046,  81968,  81295]) target tensor(30439)
input tensor([ 56442, 101734, 102200,  37046,  81968,  81295,  30439]) target tensor(55325)
input tensor([ 56442, 101734, 102200,  37046,  81968,  81295,  30439,  55325]) target tensor(38104)
